In [3]:
import os
import dotenv

dotenv.load_dotenv()

CONNECTION = os.getenv("SQLALCHEMY_STOCKS1DAY_URI")
TWELVEDATA_KEY = os.getenv("TWELVEDATA_API_KEY")


from sqlalchemy import create_engine, text

engine = create_engine(CONNECTION)
with engine.connect() as conn:
    cursor = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    print([name[0] for name in cursor].__len__())

10219


In [2]:
import requests
# stocks list
STOCK_LIST_URL = "https://api.twelvedata.com/stocks?exchange={}&apikey={}"
NYSE_STOCK_LIST_RESPONSE: dict = requests.get(STOCK_LIST_URL.format("NYSE", TWELVEDATA_KEY)).json()
NASDAQ_STOCK_LIST_RESPONSE: dict = requests.get(STOCK_LIST_URL.format("NASDAQ", TWELVEDATA_KEY)).json()
stock_symbols = set(stock["symbol"] for stock in NYSE_STOCK_LIST_RESPONSE["data"] if stock["symbol"].isalnum()) | set(stock["symbol"] for stock in NASDAQ_STOCK_LIST_RESPONSE["data"] if stock["symbol"].isalnum())
print("Num stocks: ", stock_symbols.__len__())
# etf list
ETF_LIST_URL = "https://api.twelvedata.com/etfs?exchange={}&apikey={}"
NYSE_ETF_LIST_RESPONSE: dict = requests.get(ETF_LIST_URL.format("NYSE", TWELVEDATA_KEY)).json()
NASDAQ_ETF_LIST_RESPONSE: dict = requests.get(ETF_LIST_URL.format("NASDAQ", TWELVEDATA_KEY)).json()
etf_symbols = set(stock["symbol"] for stock in NYSE_ETF_LIST_RESPONSE["data"] if stock["symbol"].isalnum()) | set(stock["symbol"] for stock in NASDAQ_ETF_LIST_RESPONSE["data"] if stock["symbol"].isalnum())
print("Num etfs: ", etf_symbols.__len__())

engine = create_engine(CONNECTION)
existing_tables = ()
with engine.connect() as conn:
    cursor = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    existing_tables = set([name[0] for name in cursor])

symbols = list((stock_symbols | etf_symbols) - existing_tables)
print("Remaining symbols: ", symbols.__len__())

Num stocks:  7252
Num etfs:  3065
Remaining symbols:  99


In [ ]:
import datetime
import pandas as pd
import dotenv
import os
import requests

symbol = "AAPL"

dotenv.load_dotenv()

TWELVEDATA_KEY = os.getenv("TWELVEDATA_API_KEY")

date_range = pd.date_range(start="2008-01-01", end=datetime.datetime.now())
date = date_range[-4].strftime("%Y-%m-%d")

url = f"https://www.alphavantage.co/query?function=HISTORICAL_OPTIONS&symbol={symbol}&date={date}&apikey={TWELVEDATA_KEY}"

response = requests.get(url).json()["data"]
df = pd.DataFrame(response)
df.size

47440

In [16]:
print(date)

2025-02-21


In [56]:
expirations = df["expiration"].unique()
print(expirations)
df.where(df["expiration"] == expirations[0]).dropna()

['2025-02-21' '2025-02-28' '2025-03-07' '2025-03-14' '2025-03-21'
 '2025-03-28' '2025-04-04' '2025-04-17' '2025-05-16' '2025-06-20'
 '2025-07-18' '2025-08-15' '2025-09-19' '2025-10-17' '2025-12-19'
 '2026-01-16' '2026-03-20' '2026-06-18' '2026-12-18' '2027-01-15'
 '2027-06-17']


,contractID,symbol,expiration,strike,type,last,mark,bid,bid_size,ask,ask_size,volume,open_interest,date,implied_volatility,delta,gamma,theta,vega,rho
0,AAPL250221C00100000,AAPL,2025-02-21,100.00,call,145.53,145.68,143.90,6,147.45,3,0,13,2025-02-21,6.91296,0.99614,0.00013,-0.52173,0.00148,0.00271
1,AAPL250221P00100000,AAPL,2025-02-21,100.00,put,0.00,0.01,0.00,0,0.01,4501,0,6508,2025-02-21,5.45507,-0.00050,0.00003,-0.06253,0.00023,-0.00000
2,AAPL250221C00105000,AAPL,2025-02-21,105.00,call,149.85,140.62,138.70,30,142.55,30,0,0,2025-02-21,6.06298,0.99772,0.00009,-0.29102,0.00092,0.00286
3,AAPL250221P00105000,AAPL,2025-02-21,105.00,put,0.00,0.01,0.00,0,0.01,500,0,2183,2025-02-21,5.17138,-0.00053,0.00003,-0.06228,0.00024,-0.00000
4,AAPL250221C00110000,AAPL,2025-02-21,110.00,call,135.49,135.62,133.85,30,137.40,30,0,0,2025-02-21,5.74378,0.99761,0.00010,-0.28783,0.00096,0.00300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,AAPL250221P00370000,AAPL,2025-02-21,370.00,put,0.00,124.62,122.90,3,126.35,2,0,0,2025-02-21,3.44368,-0.98551,0.00083,-0.76987,0.00472,-0.01004
142,AAPL250221C00375000,AAPL,2025-02-21,375.00,call,0.01,0.01,0.00,0,0.01,5,0,1480,2025-02-21,2.58776,0.00112,0.00011,-0.06196,0.00048,0.00001
143,AAPL250221P00375000,AAPL,2025-02-21,375.00,put,0.00,129.53,127.70,3,131.35,2,0,0,2025-02-21,3.30832,-0.99081,0.00058,-0.48199,0.00318,-0.01021
144,AAPL250221C00380000,AAPL,2025-02-21,380.00,call,0.01,0.01,0.00,0,0.01,4530,1,1721,2025-02-21,2.66062,0.00109,0.00011,-0.06223,0.00047,0.00001


In [31]:
# split df into calls and puts
calls = df.where(df["type"] == "call").dropna()
puts = df.where(df["type"] == "put").dropna()

calls.head()

,contractID,symbol,expiration,strike,type,last,mark,bid,bid_size,ask,ask_size,volume,open_interest,date,implied_volatility,delta,gamma,theta,vega,rho
0,AAPL250221C00100000,AAPL,2025-02-21,100.00,call,145.53,145.68,143.90,6,147.45,3,0,13,2025-02-21,6.91296,0.99614,0.00013,-0.52173,0.00148,0.00271
2,AAPL250221C00105000,AAPL,2025-02-21,105.00,call,149.85,140.62,138.70,30,142.55,30,0,0,2025-02-21,6.06298,0.99772,0.00009,-0.29102,0.00092,0.00286
4,AAPL250221C00110000,AAPL,2025-02-21,110.00,call,135.49,135.62,133.85,30,137.40,30,0,0,2025-02-21,5.74378,0.99761,0.00010,-0.28783,0.00096,0.00300
6,AAPL250221C00115000,AAPL,2025-02-21,115.00,call,113.65,130.62,128.70,6,132.55,6,0,1,2025-02-21,5.43906,0.99750,0.00011,-0.28465,0.00100,0.00313
8,AAPL250221C00120000,AAPL,2025-02-21,120.00,call,118.95,125.68,123.90,3,127.45,5,0,0,2025-02-21,5.58159,0.99530,0.00019,-0.50475,0.00176,0.00325


In [49]:
call_bids = [f"{bid} ({bid_size})" for bid, bid_size in zip(calls["bid"], calls["bid_size"])]

call_asks = [f"{ask} ({ask_size})" for ask, ask_size in zip(calls["ask"], calls["ask_size"])]

put_bids = [f"{bid} ({bid_size})" for bid, bid_size in zip(puts["bid"], puts["bid_size"])]

put_asks = [f"{ask} ({ask_size})" for ask, ask_size in zip(puts["ask"], puts["ask_size"])]


1186
1186
1186
1186
1186
1186


In [ ]:
# create options chain
options_df = pd.DataFrame({
    "Call Bid": call_bids,
    "Call Mark": list( calls["mark"]),
    "Call Ask": call_asks,
    "Strike": list(calls["strike"]),
    "Put Bid": put_bids,
    "Put Mark": list(puts["mark"]),
    "Put Ask": put_asks
    }, index=calls["expiration"])


# color the dataframe if strike is greater than 200
options_df.style.applymap(lambda x: "background-color: red" if float(x.split(" ")[0]) > 200 else "", subset=options_df.columns[2])


/tmp/ipykernel_6734/2288596593.py:20: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  options_df.style.applymap(color_strike)


KeyError: '`Styler.apply` and `.map` are not compatible with non-unique index or columns.'